#### 1 вариант
#### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии
* Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
* Ссылку на саму вакансию
* Сайт откуда собрана вакансия

#### По своему желанию можно добавить еще параметры вакансии (например работодателя и расположение). Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [0]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import lxml
import numpy as np
import unicodedata
import pandas as pd

In [3]:
main_link = 'https://hh.ru/search/vacancy'
main_link2 = 'https://russia.superjob.ru/vacancy/search'#?keywords
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
# получение пользовательского запроса
search_string = input()
# обработка запроса - приведение к виду, который используется на сайте
# hh.ru принимает такой запрос
search_query1 = '+'.join(search_string.split())
# superjob.ru принимает такой запрос
search_query2 = '%20'.join(search_string.split())
# сохраняем результат запроса в request_result

#####
# получение результатов запроса с сайта hh.ru
#####
request_result = requests.get(main_link+'?text='+search_query1, headers=header)
# bs_result - результат разбора первого запроса
bs_result = bs(request_result.text,'lxml')
# все обработанные bs() результаты для первого источника сохраняем в список bs_result
bs_result_list = []
bs_result_list.append(bs_result)
i=1
# на сайте hh.ru при нахождении определенного блока на странице можно сделать вывод, что вакансии на этой странице есть;
# поэтому сохраним такую страницу для последующего разбора
while requests.get(main_link+'?text='+search_query1+'&page='+str(i), headers=header).ok and bs(requests.get(main_link+'?text='+search_query1+'&page='+str(i), headers=header).text,'lxml').find('div',{'data-qa':'vacancy-serp__results'}).getText()!='':
  # получаем еще одну разобранную функцией bs страницу
  bs_result = bs(requests.get(main_link+'?text='+search_query1+'&page='+str(i), headers=header).text,'lxml')
  # сохраняем его в список для разбора вакансий hh.ru
  bs_result_list.append(bs_result)
  i+=1

#####
# получение результатов запроса с сайта superjob.ru
#####   
request_result2 = requests.get(main_link2+'?keywords='+search_query2, headers=header)
# получение результатов со второго источника тем же методом - и сохранение их в список bs_result2
bs_result2 = bs(request_result2.text,'lxml')
bs_result2_list = []
bs_result2_list.append(bs_result2)
# номер последней страницы можно узнать из paging внизу страницы
page_list = bs_result2.find_all('span',{'class':'_3IDf-'})
# 0 и 1 страницы на superjob - это одно и то же; поэтому, после получения первых результатов (первая страница) переходим ко второй (2)
i=2
# в получившемся page_list 9 "лишних" элементов; если элементов больше, значит, есть и другие, кроме проверенной, страницы
if any('>1</span>' in str(i) for i in page_list):
  # получение номера последней страницы
  last_page = int(str(page_list[-3:-1][0]).split('>')[1].split('<')[0])
  # в данном цикле полученные разобранные страницы сохраняются в отдельный список
  while i <= last_page:
    bs_result2 = bs(requests.get(main_link2+'?keywords='+search_query2+'&page='+str(i), headers=header).text,'lxml')
    bs_result2_list.append(bs_result2)
    i+=1

аналитик данных


In [0]:
# создаем списки для сохранения данных о зарплате
# минимальная зарплата
min_salary = []
# максимальная зарплата
max_salary = []
# валюта, в которой платится зарплата
currency = []
# список вакансий
vac_list_title=[]
vac_list_title2=[]
for bs_result in bs_result_list:
  # переходим к нужному div'у
  bs_result_vacancies=bs_result.find('div',{'class':'vacancy-serp'})
  for i in bs_result_vacancies.find_all('a',{'data-qa':'vacancy-serp__vacancy-title'}):
      link = main_link+i.get('href')
      vac_list_title.append([link,i.getText(),'HeadHunter'])
  # для каждого элемента в получающемся списке проверям 4 условия
  # 1 - если строка с информацией о зарплате начинается со слова "до": создаем список данных разбиением по пробелу;
  # "до" удаляем; последний элемент списка = валюта; все, что до него - зарплата
  # 2 - та же логика
  # 3 - если в тексте строки есть знак '-', значит, указана зарплата "от и до":
  # первый элемент - это минимальная зарплата;
  # второй - максимальная;
  #последний элемент после сплита максимальной зарплаты по пробелу - это название валюты
  # 4 - последний вариант: если ничего не указано, значит, данных нет.
  # np.nan для списков зарплаты, пустая строка - для списка валют.
  for i in bs_result_vacancies.find_all('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'}):
      vac_salary = i.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
      #print(0,vac_salary)
      if vac_salary:
      # избавляемся от ненужных символов
          salary = unicodedata.normalize('NFKD',vac_salary.getText())
          #print(1,salary)
          if salary[0:2]=='до':
              #print(2)
              max_salary.append(''.join(salary.split(' ')[:-1]).replace('до',''))
              currency.append(salary.split(' ')[-1])
              min_salary.append(np.nan)
          elif salary[0:2]=='от':
              #print(3)
              min_salary.append(''.join(salary.split(' ')[:-1]).replace('от',''))
              currency.append(salary.split(' ')[-1])
              max_salary.append(np.nan)
          elif '-' in salary:
              #print(4)
              min_salary.append(''.join(salary.split('-')[0].split(' ')))
              currency.append(salary.split('-')[-1].split(' ')[-1])
              max_salary.append(''.join(salary.split('-')[1].split(' ')[:-1]))
      else:
          #print(6,vac_salary,'+')
          min_salary.append(np.nan)
          max_salary.append(np.nan)
          currency.append('')
  #print(*min_salary,*max_salary,*currency)
  #print(len(min_salary),len(max_salary),len(currency))
# все подобные операции проводим по второму запросу
for bs_result2 in bs_result2_list:
  # по какой-то причине теги на superjob меняются; поэтому обрабатываются сразу 2 варианта, которые были выявлены при анализе сайта
  try:
    bs_result_vacancies2=bs_result2.find_all('div',{'class':'f-test-vacancy-item'})
    #bs_result_vacancies2=bs_result2.find_all('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
  except Exception:
    pass
  # если после первой попытки в списке bs_result_vacancies2 ничего нет, запускается второй вариант
  if len(bs_result_vacancies2)==0:
    try:
      bs_result_vacancies2=bs_result2.find_all('div',{'class':'_3syPg _3P0J7 _9_FPy'})
    except Exception:
      pass
  # информация о зарплате хранится отдельно - получаем этот кусочек кода
  salaries=bs_result2.find_all('span',{'class':'f-test-text-company-item-salary'})
  for i in bs_result_vacancies2:
    try:
      link2 = main_link2+i.find('a',{'class':'icMQ_'}).get('href')
      vac_list_title2.append([link2,i.find('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText(),'SuperJob'])
    except Exception:
      link2 = main_link2+i.find('a',{'class':'icMQ_'}).get('href')
      vac_list_title2.append([link2,i.find('span',{'class':'_1rS-s'}).getText(),'SuperJob'])
  # данные о заработных платах и валютах добавляются в ранее созданные и уже заполненные первым источником списки
  for vac_salary2 in salaries:
      vac_salary2 = unicodedata.normalize("NFKD",vac_salary2.getText())
      if vac_salary2[0:3]=='до ':
          max_salary.append(''.join(vac_salary2.split(' ')[:-1]).replace('до',''))
          currency.append(vac_salary2.split(' ')[-1])
          min_salary.append(np.nan)
      elif vac_salary2[0:3]=='от ':
          min_salary.append(''.join(vac_salary2.split(' ')[:-1]).replace('от',''))
          currency.append(vac_salary2.split(' ')[-1])
          max_salary.append(np.nan)
      elif '—' in vac_salary2:
          min_salary.append(''.join(vac_salary2.split('—')[0].split(' ')))
          currency.append(vac_salary2.split('—')[-1].split(' ')[-1])
          max_salary.append(''.join(vac_salary2.split('—')[1].split(' ')[:-1]))
      elif vac_salary2[:10] == 'По договор':
          min_salary.append(np.nan)
          max_salary.append(np.nan)
          currency.append('')
      else:
          min_salary.append(vac_salary2)
          max_salary.append(vac_salary2)
          currency.append('')
# вся полученная информация о зарплате сохраняется в отдельный датафрейм; из него она будет добавлена в dataframe result
salary_data=pd.DataFrame([min_salary,max_salary,currency]).T.rename(columns = {0:'min_salary',1:'max_salary',2:'currency'})
# результирующий список вакансий
result_list = []

# список зарплат = salary_data
# список вакансий = vac_list_title + vac_list_title2
vac_list_title.extend(vac_list_title2)
# определяем длину списка вакансий - это общее количество собранных вакансий
i=len(vac_list_title)
for k in range(i):
    result_list.append([vac_list_title[k][1],\
                        salary_data['min_salary'][k],\
                        salary_data['max_salary'][k],\
                        salary_data['currency'][k],\
                        vac_list_title[k][0],\
                        vac_list_title[k][2]])#.split('.ru')[0].split('https://')[-1]+'.ru'
result = pd.DataFrame(result_list).rename(columns={0:'Вакансия',1:'Мин.зарплата',2:'Макс.зарплата',3:'Валюта',4:'Ссылка',5:'Сервис'})

In [5]:
result

,Вакансия,Мин.зарплата,Макс.зарплата,Валюта,Ссылка,Сервис
0,Аналитик данных (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
1,Аналитик данных в команду рисков (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
2,Аналитик данных / POS (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
3,Маркетолог-аналитик,80000,140000,руб.,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
4,Аналитика данных / POS 2.0 (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
...,...,...,...,...,...,...
2210,Системный архитектор,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2211,Менеджер по интернет-маркетингу / SEO-менеджер...,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2212,Главный специалист отдела согласования и выдач...,NaN,55000,₽,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2213,Менеджер по закупкам и работе с маркетплейсами,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob


In [0]:
# обработка нецифровых значений в поле минимальной / максимальной зарплаты
for k,i in enumerate(result['Мин.зарплата']):
  try:
    result['Мин.зарплата'][k]=float(result['Мин.зарплата'][k])
  except Exception:
    x=[]
    ii = list(result['Мин.зарплата'][k])
    while type(i) != int:
      iii = ii.pop(-1)
      x.append(iii)
      try:
        i = int(''.join(ii))
      except Exception:
        pass
    result['Мин.зарплата'][k]=float(i)
    result.iloc[k,3] = ''.join(list([result['Валюта'][k],''.join(x[::-1])])[::-1])

for k,i in enumerate(result['Макс.зарплата']):
  try:
    result['Макс.зарплата'][k]=float(result['Макс.зарплата'][k])
  except Exception:
    x=[]
    ii = list(result['Макс.зарплата'][k])
    while type(i) != int:
      iii = ii.pop(-1)
      x.append(iii)
      try:
        i = int(''.join(ii))
      except Exception:
        pass
    result['Макс.зарплата'][k]=float(i)
    result.iloc[k,3] = ''.join(list([result['Валюта'][k],''.join(x[::-1])])[::-1])
# смена типа столбцов в датафрейме
result['Мин.зарплата'] = result['Мин.зарплата'].astype(float)
result['Макс.зарплата'] = result['Макс.зарплата'].astype(float)
# обработка разных обозначений валют; пока по-разному обозначена только российская валюта
result.loc[result['Валюта']=='₽','Валюта']='руб.'

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 6 columns):
Вакансия         2215 non-null object
Мин.зарплата     599 non-null float64
Макс.зарплата    375 non-null float64
Валюта           2215 non-null object
Ссылка           2215 non-null object
Сервис           2215 non-null object
dtypes: float64(2), object(4)
memory usage: 104.0+ KB


In [8]:
# NaN - нет данных; это np.nan, а не строка
result

,Вакансия,Мин.зарплата,Макс.зарплата,Валюта,Ссылка,Сервис
0,Аналитик данных (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
1,Аналитик данных в команду рисков (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
2,Аналитик данных / POS (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
3,Маркетолог-аналитик,80000.0,140000.0,руб.,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
4,Аналитика данных / POS 2.0 (стажер),NaN,NaN,,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,HeadHunter
...,...,...,...,...,...,...
2210,Системный архитектор,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2211,Менеджер по интернет-маркетингу / SEO-менеджер...,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2212,Главный специалист отдела согласования и выдач...,NaN,55000.0,руб.,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
2213,Менеджер по закупкам и работе с маркетплейсами,NaN,NaN,,https://russia.superjob.ru/vacancy/search/vaka...,SuperJob
